In [2]:
import math
from math import comb
import numpy as np

In [3]:
def count_card_sum_ways():
    """
    Compute the number of ways of obtaining any given sum choosing 10 cards from 4 (4 suits)
    and cards numbered 1 - 10. The minimum and maximum sums are:
    min_sum = 4*1 + 4*2 + 2*3  = 18
    max_sum = 4*10 + 4*9 + 2*8 = 92
    Due to replacement. 
    return -> List: index is the sum with index 0 corresponding to obtaining 18. The value is 
    all the possible ways of obtaining that sum.
    """
    max_cards = 10
    max_sum = 100
    
    # dp[cards_used][sum] = number of ways
    dp = [[0] * (max_sum + 1) for _ in range(max_cards + 1)]
    dp[0][0] = 1
    
    for card_value in range(1, 11):
        new_dp = [[0] * (max_sum + 1) for _ in range(max_cards + 1)]
        
        # Copy existing states
        for cards in range(max_cards + 1):
            for sum_val in range(max_sum + 1):
                new_dp[cards][sum_val] = dp[cards][sum_val]
        
        # Add states using this card value (up to 4 times)
        for use_count in range(1, 5):
            for cards in range(max_cards + 1):
                for sum_val in range(max_sum + 1):
                    if dp[cards][sum_val] > 0 and cards + use_count <= max_cards:
                        new_cards = cards + use_count
                        new_sum = sum_val + card_value * use_count
                        if new_sum <= max_sum:
                            ways_to_choose = comb(4, use_count)
                            new_dp[new_cards][new_sum] += dp[cards][sum_val] * ways_to_choose
        
        dp = new_dp
    
    # Return results for exactly 10 cards, the range is given by
    min_sum = 4*1 + 4*2 + 2*3  # 18
    max_sum = 4*10 + 4*9 + 2*8  # 92
    
    return [dp[10][sum_val] for sum_val in range(min_sum, max_sum + 1)]

# Get the final array
result = count_card_sum_ways()

In [4]:
final_price = [i for i in range(18, 93)]
payoffs = [max(i-55,0) for i in final_price]
all_possible_combinations = sum(result)
sanity_check = math.comb(40,10) # Total number of ways of choosing 10 cards
all_possible_combinations == sanity_check

True

In [5]:
import numpy as np
probs = [j / all_possible_combinations for j in result]
ev = np.dot(probs, payoffs)
print("The EV of the game with no knowledge: {ev}")

The EV of the game with no knowledge: {ev}


In [6]:
def compute_ev_given_card(given_card):
    """
    Do: count_card_sum_ways given that we know one card. Like in the game setting.
    return -> List: index is the sum with index 0 corresponding to obtaining 18. The value is 
    all the possible ways of obtaining that sum.
    """
    max_cards = 9
    max_sum = 92
    
    dp = [[0] * (max_sum + 1) for _ in range(max_cards + 1)]
    dp[0][0] = 1
    
    for card_value in range(1, 11):
        new_dp = [[0] * (max_sum + 1) for _ in range(max_cards + 1)]
        
        for cards in range(max_cards + 1):
            for sum_val in range(max_sum + 1):
                new_dp[cards][sum_val] = dp[cards][sum_val]
        
        max_use = 3 if card_value == given_card else 4
        max_use = min(max_use, max_cards)
        
        for use_count in range(1, max_use + 1):
            for cards in range(max_cards + 1):
                for sum_val in range(max_sum + 1):
                    if dp[cards][sum_val] > 0 and cards + use_count <= max_cards:
                        new_cards = cards + use_count
                        new_sum = sum_val + card_value * use_count
                        if new_sum <= max_sum:
                            available = 3 if card_value == given_card else 4
                            ways_to_choose = comb(available, use_count)
                            new_dp[new_cards][new_sum] += dp[cards][sum_val] * ways_to_choose
        
        dp = new_dp
    
    results = {}
    for sum_val in range(max_sum + 1):
        if dp[9][sum_val] > 0:
            final_sum = sum_val + given_card
            results[final_sum] = results.get(final_sum, 0) + dp[9][sum_val]
    
    total_combinations = sum(results.values())
    final_price = list(range(min(results.keys()), max(results.keys()) + 1))
    counts = [results.get(s, 0) for s in final_price]
    probs = [c / total_combinations for c in counts]
    payoffs = [max(s - 55, 0) for s in final_price]
    
    return np.dot(probs, payoffs)

In [7]:
ev_10 = compute_ev_given_card(10)

In [8]:
ev_10

5.04631245493125

In [9]:
for i in range(1,11):
    print(f"The expected value of the game given the card: {i} is {compute_ev_given_card(i)}")

The expected value of the game given the card: 1 is 1.5847739933927887
The expected value of the game given the card: 2 is 1.8854275399266913
The expected value of the game given the card: 3 is 2.208775563040019
The expected value of the game given the card: 4 is 2.5533024560039435
The expected value of the game given the card: 5 is 2.918090068244867
The expected value of the game given the card: 6 is 3.3027054528602515
The expected value of the game given the card: 7 is 3.7071486098500976
The expected value of the game given the card: 8 is 4.1318524861169434
The expected value of the game given the card: 9 is 4.577735232234384
The expected value of the game given the card: 10 is 5.04631245493125


In [15]:
def sanity_check():
    t_sum = 0
    for i in range(1,11):
        t_sum += compute_ev_given_card(i)
    t_sum /= 10
    return t_sum

ev_without_knowledge = sanity_check()
ev_without_knowledge - ev < 1e-15

True